In [110]:
import pandas as pd
import joblib
import numpy as np
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split,GridSearchCV

In [111]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

In [112]:
O_E = OrdinalEncoder()
train[['id','Target']] = O_E.fit_transform(train[['id','Target']])
train.head(2)

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0.0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,2.0
1,1.0,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,0.0


In [ ]:
duplicates = train[train.duplicated()]
print(duplicates)

In [ ]:
train.corr()['Target'].sort_values(ascending=False)

Drop clounms that has no correlation with indipendent variable
1 Educational special needs  
2 International
3 Nacionality
4 Unemployment rate
5 Curricular units 1st sem (credited) 
6 Curricular units 2nd sem (credited)
7 Mother's qualification
8 Previous qualification
9 Father's qualification
10 Father's occupation  
11 Inflation rate 

In [ ]:
train = train.drop(columns=["Educational special needs", "International", "Nacionality", "Unemployment rate", "Curricular units 1st sem (credited)", 
                            "Curricular units 2nd sem (credited)", "Mother's qualification", "Previous qualification", "Father's qualification" 
                            ,"Father's occupation", "Inflation rate"])

In [ ]:
test = test.drop(columns=["Educational special needs", "International", "Nacionality", "Unemployment rate", "Curricular units 1st sem (credited)", 
                            "Curricular units 2nd sem (credited)", "Mother's qualification", "Previous qualification", "Father's qualification" 
                            ,"Father's occupation", "Inflation rate"])

In [ ]:
Q1 = train.quantile(0.25)
Q3 = train.quantile(0.75)
IQR = Q3 - Q1

In [ ]:
outliers = ((train < (Q1 - 1.5 * IQR)) | (train > (Q3 + 1.5 * IQR)))
print(outliers)

In [ ]:
train = train[~((train < (Q1 - 1.5 * IQR)) | (train > (Q3 + 1.5 * IQR))).any(axis=1)]
print(train)

In [ ]:
print(train.describe())

In [129]:
model = CatBoostClassifier()

In [130]:
features= train.drop(['id', 'Target'],axis=1)
target=train['Target']

In [131]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [145]:
model.fit(X_train, y_train)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostClassifier object at 0x00000270B2613740>,
             n_jobs=-1,
             param_grid={'depth': [4, 6, 8], 'iterations': [100, 200],
                         'l2_leaf_reg': [1, 3, 5, 7, 9],
                         'learning_rate': [0.01, 0.05, 0.1]},
             scoring='accuracy', verbose=1)

In [147]:
testset = test.drop('id' , axis =1)

In [148]:
predt = model.predict(X_test)

In [150]:
pred = model.predict(testset)

In [149]:
accuracy = accuracy_score(y_test, predt)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 83.21%


In [153]:
predset = pd.DataFrame({'id': test['id'], 'Target': pred})
predset.head(3)

,id,Target
0,76518,0.0
1,76519,2.0
2,76520,2.0


In [154]:
def targetNames(col):
    if col == 0:
        return 'Dropout'
    if col == 1:
        return 'Enrolled'
    if col == 2:
        return 'Graduate'

prediction_df = pd.Series(pred).apply(targetNames)
preddf = pd.DataFrame({'id': test['id'], 'Target': prediction_df})
preddf.head(10)

,id,Target
0,76518,Dropout
1,76519,Graduate
2,76520,Graduate
3,76521,Enrolled
4,76522,Enrolled
5,76523,Graduate
6,76524,Graduate
7,76525,Graduate
8,76526,Dropout
9,76527,Graduate


In [155]:
preddf.to_csv('sample_submission.csv', index=False)

In [ ]:
accuracy_score(sampledf,pred) 